In [1]:
import pymc as pm
import numpy as np
import arviz as az
import matplotlib.pyplot as plt

In [2]:
# ASSUMPTIONS

# Assumptions for ticket revenues
arena_capacity = 20000
total_games = 41
ticket_price = 200
base_fill_rate = 0.7

# LeBron uplift assumptions
lebron_war = 11.5                          # Wins above replacement. LeBron is directly responsible for a delta of 11.5 more wins than baseline
lebron_uplift_factor = lebron_war / 82     # LeBron provides a 14% extra chance to win any given game

# Fixed costs and revenues
rest_of_team_salaries = 155000000
rest_of_team_merch = 18300000
other_profit = 25000000
base_sales = 1200000 * 111

In [3]:
def scenario_simulation(acquire_lebron, market_research, request_gambling_data, market_research_results, gambling_data_results):

  # Define the additional costs that come from the decisions (which are 1s or 0s)
  cost_gambling_intel = request_gambling_data * 1000000
  cost_mkt_research = market_research * 1000000
  lebron_contract = acquire_lebron * 52000000

  model = pm.Model()

  with model:

    # Popularity and market research on popularity
    popularity = pm.Beta("Popularity", alpha = 5, beta = 3)
    if market_research == 1:
      market_research_results = pm.Bernoulli("Market_Research_Results", p = popularity, observed = market_research_results)
    else:
      market_research_results = pm.Bernoulli("Market_Research_Results", p = popularity)

    # Gambling and data on gambling
    gambling = pm.Bernoulli("Gambling", p = 0.6)
    caught = pm.Bernoulli("Caught", p = pm.math.switch(gambling, 0.9, 0))
    if request_gambling_data == 1:
      investigation_result = pm.Bernoulli("Investigation_Result", p = pm.math.switch(gambling, 0.9, 0.01), observed = gambling_data_results)
    else:
      investigation_result = pm.Bernoulli("Investigation_Result", p = pm.math.switch(gambling, 0.9, 0.01))

    # Games missed and played by LeBron (if lebron is not acquired he has 0 games played by default, see the "acquire_lebron *" operator)
    lebron_games_missed = pm.Gamma("LeBron_Games_Missed", alpha = 2, beta = 1/5)
    lebron_games_played = pm.Deterministic("LeBron_Games_Played", acquire_lebron * (82 - lebron_games_missed))

    # Team performance and games won
    team_perf = pm.Uniform("Team_Perf", lower = 0.8, upper = 1.2)
    expected_games_won = (35 * team_perf) + (lebron_games_played * lebron_uplift_factor)
    games_won = pm.Normal("Games_Won", mu = expected_games_won, sigma = 5)

    # Calculate the value of LeBron jersey sales (if lebron is not acquired he has 0 jersey sales by default, see the "acquire_lebron *" operator)
    expected_lebron_merch_sales = base_sales * (0.5 + popularity) * (0.2 + (lebron_games_played / 82))
    lebron_jersey_sales_distribution = pm.Normal("LeBron_Jersey_Sales_Distribution", mu = expected_lebron_merch_sales, sigma = expected_lebron_merch_sales * 0.1)
    lebron_jersey_sales = pm.Deterministic("LeBron_Jersey_Sales", acquire_lebron * lebron_jersey_sales_distribution)

    # Ticket sales considering if LeBron was acquired or not
    gw_factor = 1 + (0.39/(1 + pm.math.exp((-0.155 * (games_won - 37.8)))) - 0.2)
    lebron_gp_factor = 1 + (0.194/(1 + pm.math.exp((-0.155 * (lebron_games_played - 37.8)))) - 0.1)
    lebron_pop_factor = 1 + (0.05/(1 + pm.math.exp((-8.61 * (popularity - 0.4)))) - 0.0007)
    if acquire_lebron == 0:
      expected_ticket_sales = base_fill_rate * gw_factor * arena_capacity * total_games * ticket_price
    else:
      expected_ticket_sales = base_fill_rate * gw_factor * lebron_gp_factor * lebron_pop_factor * arena_capacity * total_games * ticket_price
    ticket_sales = pm.Normal("Ticket_Sales", mu = expected_ticket_sales, sigma = 0.1 * expected_ticket_sales)

    # Probability of winning the championship
    lambda_rate = 312 * (pm.math.exp(-0.11 * games_won))
    seed = pm.Poisson("Seed", mu = lambda_rate) + 1        # Position in final standings
    win_prob = pm.math.switch(
      pm.math.eq(seed, 1), 0.4,
      pm.math.switch(
        pm.math.eq(seed, 2), 0.3,
        pm.math.switch(
          pm.math.eq(seed, 3), 0.25,
          pm.math.switch(
            pm.math.and_(seed >= 4, seed <= 6), 0.1,
            pm.math.switch(
              pm.math.or_(pm.math.eq(seed, 7), pm.math.eq(seed, 8)), 0.02, 0)
            )
          )
        )
      )
    win_championship = pm.Bernoulli("Win_Championship", p = win_prob)

    # Define the profit from all cash flows from the season
    total_merch_sales = pm.Deterministic("Total_Merch_Sales", lebron_jersey_sales - caught * lebron_jersey_sales)
    total_revenue = pm.Deterministic("Total_Revenue", ticket_sales + total_merch_sales)
    profit = total_revenue + rest_of_team_merch + other_profit - lebron_contract - rest_of_team_salaries
    profit_util_function = 1 / (1 + np.exp((-1 * (profit - 35000000)) / 25000000))

    # Define the utility from all the outcomes
    utility_profit = pm.Deterministic("Profit_Utility", profit_util_function)
    utility_winning = pm.Deterministic("Winning_Utility", win_championship)
    total_utility = pm.Deterministic("Total_Utility", 0.6 * utility_profit + 0.18 * utility_winning + 0.22 * utility_profit * utility_winning)

  return model

In [4]:
# Define all possible reasonable combinations of 0s and 1s
# An unreasonable combination is doing research, getting good results and not hiring LeBron

cases = [[0, 0, 0, None, None],       # Don't do any research and not hire
         [1, 0, 0, None, None],       # Don't do any research and hire
         [0, 0, 1, None, 1],          # Only do gambling investigation and don't hire if he is gambling
         [1, 0, 1, None, 0],          # Only do gambling investigation and hire if he is not gambling
         [0, 1, 0, 0, None],          # Only do popularity research and not hire if he is not popular
         [1, 1, 0, 1, None]]          # Only do popularity research and hire if he is popular

models = [scenario_simulation(*c) for c in cases]

In [16]:
market_research_results_mean = []
investigation_result_mean = []
expected_utilities = []
for model in models:
  with model:
    trace = pm.sample(draws=10000, tune=1000, chains=2, random_seed=0)                                          # Get the trace of the model
    expected_utilities.append((trace['posterior']['Total_Utility']).to_numpy().mean())                          # Expected utility given parameter inputs
    try:
      market_research_results_mean.append((trace['posterior']['Market_Research_Results']).to_numpy().mean())  # Average times market research would return 1
    except:
      market_research_results_mean.append('NULL')
    try:
      investigation_result_mean.append((trace['posterior']['Investigation_Result']).to_numpy().mean())        # Average times gambling investigation would return 1
    except:
      investigation_result_mean.append('NULL')

Output()

Output()

ERROR:pymc.stats.convergence:There were 1 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

Output()

...

Output()

ERROR:pymc.stats.convergence:There were 33 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

Output()

ERROR:pymc.stats.convergence:There were 741 divergences after tuning. Increase `target_accept` or reparameterize.


In [19]:
expected_utilities

[0.14142347278926187,
 0.3749222460137876,
 0.13493780761011817,
 0.5890443801654299,
 0.13931019997971256,
 0.38870164344624375]

In [22]:
# Get the probability of market research returning 1
np.mean(market_research_results_mean[:2])

0.6251

In [23]:
# Get the probability of gambling research returning 1
np.mean(investigation_result_mean[:2])

0.519525